In [ ]:
import os

from sqlalchemy import and_, or_
import pandas as pd
pd.set_option("display.max_rows", None)

from gresq.config import Config
from gresq.database.v1_1_0 import dal as db1     # prod dal
from gresq.database import dal as db2            # test dal
from gresq.database.v1_1_0 import Base as Base1  # prod base
from gresq.database import Base as Base2         # test base

from gresq.database.v1_1_0.models import Sample,Recipe



In [ ]:
# Setup old schema (production db) as db1 using read only
config_prefix = "PROD_DATABASE"
config_suffix = "_READ"
conf = Config(prefix=config_prefix, suffix=config_suffix, debug=True, try_secrets=False)
db1.init_db(conf, privileges={"read": True, "write": False, "validate": False})
# Testing Configuration and dal initialization
config_prefix = "TEST_DATABASE"
config_suffix = "_ADMIN" 
conf = Config(prefix=config_prefix, suffix=config_suffix, debug=True, try_secrets=False)
db2.init_db(conf, privileges={"read": True, "write": True, "validate": True})

# 0) Stand up fresh empty db2
ses2 = db2.Session()
ses2.close()
Base2.metadata.drop_all(bind=db2.engine)
Base2.metadata.create_all(bind=db2.engine)

In [ ]:
ses1 = db1.Session()
pd.read_sql(
    ses1.query(Sample)
    .statement,
    ses1.bind,
)

In [ ]:
# Test query for furnace data

ses1 = db1.Session()
pd.read_sql(
    ses1.query(Recipe.tube_diameter, Recipe.cross_sectional_area, Recipe.tube_length)
    .distinct()
    .filter(
        or_(
            Recipe.tube_diameter != None,
            Recipe.cross_sectional_area != None,
            Recipe.tube_length != None,
        )
    )
    .statement,
    ses1.bind,
)

In [ ]:
# Read all unique authors from old schema
from gresq.database.v1_1_0.models import Author

ses1 = db1.Session()
pd.read_sql(
    ses1.query(
        Author.sample_id,Author.first_name, Author.last_name, Author.institution, Sample.experiment_date
    ).filter(Author.sample_id==Sample.id)
    .group_by(Author.sample_id,Author.first_name, Author.last_name)
    .statement,
    ses1.bind,
)

In [ ]:
ses1 = db1.Session
authors = ses1.query(
    Author.sample_id, Author.first_name, Author.last_name, Author.institution
).group_by(Author.first_name, Author.last_name)
ses1.query(Sample.id,Sample.recipe.nanohub_id).filter(Sample.id.in_(authors))

In [ ]:
from gresq.database.v1_1_0.models import Author
from gresq.database.models import Author as Author2
from gresq.database.models import Furnace as Furnace2
commit = False

# Some migration steps

# 1) Get unique authors from old prod db
ses1 = db1.Session()
# authors = ses1.query(Author.first_name, Author.last_name, Author.institution).distinct()

authors = ses1.query(
    Author.sample_id, Author.first_name, Author.last_name, Author.institution
).group_by(Author.first_name, Author.last_name)


# 2) Get unique furnace values
furnaces = ses1.query(
    Recipe.tube_diameter, Recipe.cross_sectional_area, Recipe.tube_length
).distinct()

if commit:
    # Create new Authors records
    new_authors = [
        Author2(first_name=a.first_name, last_name=a.last_name, institution=a.institution)
        for a in authors
    ]
    with db2.session_scope(autocommit=True) as ses2:
        ses2.add_all(new_authors)
        ses2.add_all(furnaces)

In [ ]:
%env